SET THINGS UP

In [ ]:
import os
import time
import google.generativeai as genai #call the gemini api
from tqdm.notebook import tqdm #for progress bar

In [ ]:
# --- SETTINGS ---

#Gemini configuration
with open('gemini.txt', 'r') as f: # remember to put your API key in the gemini.txt file.
    my_api_key = f.read()
my_model = "gemini-2.5-flash" # select your prefered model, you can check the available models here: https://ai.google.dev/gemini-api/docs/models
# Set up the API
genai.configure(api_key=my_api_key)
model = genai.GenerativeModel(my_model)

#Path: remember to change the path of the folder where your store procedures are, and the path of the output folder where you want to save the documentation files.
sp_location = "[LOCATION_1]"
docutentation_output_location = "[LOCATION_2]"

SET FUNCTIONS

In [ ]:
# --- FUNCTION TO SEND THE STORE PROCEDURE TO GEMINI AND GENERATE THE DOCUMENTATION ---
def generate_documentation(sql_code,file_name):
    """
    Envía el código SQL al LLM y retorna el bloque de comentarios.
    """
    
    prompt = f"""
        Role: You are a Senior DBA expert in T-SQL and financial consolidation processes.
        Task: Generate standardized technical documentation for the following Stored Procedure.
        
        Formatting Instructions:
        1. DO NOT include greetings, introductions, or filler text ("Here is your analysis...").
        2. DO NOT use code blocks (```markdown, ```text, etc.). Return direct plain text.
        3. Strictly follow the template provided below.
        4. If a field does not apply, write "N/A".
        
        Analysis Instructions:
        - Category: Choose ONLY ONE from this list: [Amount Consolidation, Amount Validation/Comparison, Local-to-Dollar Conversion, Dollar-to-Sol Conversion, Others].
        - Description: Focus on the business logic (the "why" and "what for"); do not describe the SQL code line by line.
        - Parameters: Indicate the name, data type, and briefly explain its use. If you detect it is not used in the body of the script, mark it as "(UNUSED)".
        - Tables: Clearly separate tables that are only read (Source) and tables that are modified (Target: INSERT, UPDATE, DELETE, MERGE).
        - Executed Stored Procedures: Indicate if other stored procedure(s) are executed during the process.
        
        --- TEMPLATE START ---
        File Name: {file_name}
        Category: [CATEGORY_HERE]
        ----------------------------------------
        Process Description:
        [BUSINESS_SUMMARY_HERE]
        ----------------------------------------
        Parameters:
        - @Parameter1 (Type): Explanation...
        - @Parameter2 (Type): Explanation... (UNUSED)
        ----------------------------------------
        Affected Tables:
        [READ]: TableA, TableB
        [WRITE]: TableC
        ----------------------------------------
        Additional Stored Procedures:
        - SP #1...
        - SP #2...
        --- TEMPLATE END ---
        
        SQL Code to analyze:
        {sql_code}
        """
    
    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"/* ERROR DURING DOCUMENTATION: {str(e)} */"

In [ ]:
def process_files(list_of_files):
    print(f"{len(list_of_files)} files found. Initiating processing...")

    # We use tqdm to have a comfortable progress bar
    for name_of_file in tqdm(list_of_files):
        full_path = os.path.join(sp_location, name_of_file)
        
        with open(full_path, 'r') as f: #ensure to add the right encoding if your files have special characters, e.g., encoding='utf-8'
            codigo_sp = f.read()

        # Generate docs
        doc_header = generate_documentation(codigo_sp,name_of_file)

        #path file output
        path_new_file = docutentation_output_location+"DOCU__"+name_of_file+".txt"
        
        with open(path_new_file, "w", encoding='utf-8') as new_file:
            new_file.write(doc_header)
        
        # IMPORTANT: Pause to avoid Rate Limit issues
        time.sleep(5)

PROCESS THE FILES

In [ ]:
# Retrieve the list of SQL files in the specified folder
l_files = [f for f in os.listdir(sp_location) if f.endswith('.sql')]

# If you want to test with a smaller number of files, you can uncomment the following line to select only a subset of the files (e.g., the first 10 files):
# archivos = archivos[:10]

#Print the list of files that will be processed
l_files

In [ ]:
#You can check the number of files that will be processed with the following line:
print("Number of files to process:", len(l_files))

In [ ]:
# Finally, call the function to process the files
process_files(l_files)

CHECK FOR ERRORS IN THE STORE PROCEDURE FILES

In [ ]:
#Use this code to check for possible errors when reading the files, for example, due to encoding issues. It will try to read all the files and will return a list of the files that could not be read.
errors = []

for file_name in tqdm(l_files):
    full_path = os.path.join(sp_location, file_name)

    try:
        with open(full_path, 'r') as f: #add the encoding you wish to test, e.g., encoding='utf-16'
            a = f.read()
    except Exception as e:
        errors.append(file_name)

print("Number of files with errors: "+str(len(errors)))
print(errors)